# CompI - Getting Started

Welcome to CompI (Compositional Intelligence)! This notebook will guide you through the basic setup and first experiments.

## What is CompI?

CompI is a multi-modal AI system that combines:
- Text analysis and emotion detection
- Image generation with Stable Diffusion
- Audio processing and generation
- Creative content fusion


In [ ]:
# Setup and imports
import sys
from pathlib import Path

# Add project root to path
project_root = Path().absolute().parent
sys.path.append(str(project_root))

# Basic imports
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

# Project imports
from src.config import *
from src.utils import setup_logger

# Setup logger
logger = setup_logger("compi_notebook")
logger.info("CompI notebook started!")

## 1. Environment Check

Let's verify that everything is working correctly:

In [ ]:
# Check PyTorch and CUDA
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

## 2. Text Analysis

Let's start with basic text analysis and emotion detection:

In [ ]:
from textblob import TextBlob
from transformers import pipeline

# Initialize emotion classifier
emotion_classifier = pipeline(
    "text-classification",
    model="j-hartmann/emotion-english-distilroberta-base",
    device=0 if torch.cuda.is_available() else -1
)

# Test text
test_text = "I'm feeling excited about this new creative project!"

# Sentiment analysis with TextBlob
blob = TextBlob(test_text)
sentiment = blob.sentiment

# Emotion detection
emotions = emotion_classifier(test_text)

print(f"Text: {test_text}")
print(f"Sentiment - Polarity: {sentiment.polarity:.3f}, Subjectivity: {sentiment.subjectivity:.3f}")
print(f"Emotion: {emotions[0]['label']} (confidence: {emotions[0]['score']:.3f})")

## 3. Simple Image Generation

Now let's try generating an image based on text:

In [ ]:
# Note: This will download the Stable Diffusion model on first run (~4GB)
# Make sure you have enough disk space and a good internet connection

from diffusers import StableDiffusionPipeline

# Load the model (this may take a while on first run)
print("Loading Stable Diffusion model...")
pipe = StableDiffusionPipeline.from_pretrained(
    STABLE_DIFFUSION_MODEL,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
)
pipe = pipe.to(device)

# Enable memory efficient attention if available
if hasattr(pipe, "enable_attention_slicing"):
    pipe.enable_attention_slicing()

print("Model loaded successfully!")

In [ ]:
# Generate an image
prompt = "a serene landscape with floating islands, digital art, highly detailed"

print(f"Generating image for prompt: {prompt}")

# Generate
with torch.autocast(device.type if device.type != 'mps' else 'cpu'):
    image = pipe(
        prompt,
        num_inference_steps=DEFAULT_INFERENCE_STEPS,
        guidance_scale=DEFAULT_GUIDANCE_SCALE,
        height=DEFAULT_IMAGE_SIZE[0],
        width=DEFAULT_IMAGE_SIZE[1]
    ).images[0]

# Display the image
plt.figure(figsize=(8, 8))
plt.imshow(image)
plt.axis('off')
plt.title(f"Generated Image: {prompt}")
plt.show()

# Save the image
from src.utils import save_image
saved_path = save_image(image, "test_generation.png")
print(f"Image saved to: {saved_path}")

## 4. Next Steps

Congratulations! You've successfully:
- ✅ Set up the CompI environment
- ✅ Analyzed text for sentiment and emotion
- ✅ Generated your first AI image

### What's Next?

1. **Experiment with different prompts** - Try various artistic styles, emotions, and concepts
2. **Combine modalities** - Use emotion analysis to influence image generation
3. **Add audio** - Generate music or sounds that match the visual content
4. **Build a UI** - Create a Streamlit or Gradio interface for easy interaction

### Useful Resources:

- [Stable Diffusion Prompting Guide](https://prompthero.com/stable-diffusion-prompt-guide)
- [Hugging Face Transformers Documentation](https://huggingface.co/docs/transformers)
- [CompI Project Documentation](../README.md)


In [ ]:
# Clean up GPU memory
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    
print("🎉 Notebook completed successfully!")